In [9]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
df_call = pd.read_csv('/content/OPTSTK_RELIANCE_CE_26-Sep-2024_TO_26-Dec-2024.csv')
df_put = pd.read_csv('/content/OPTSTK_RELIANCE_PE_26-Sep-2024_TO_26-Dec-2024.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/OPTSTK_RELIANCE_CE_26-Sep-2024_TO_26-Dec-2024.csv'

In [ ]:
# Importing the yfinance library to fetch stock market and financial data
import yfinance as yf

# Using the `download` method to fetch historical data for the stock symbol 'RELIANCE.NS'
# - 'RELIANCE.NS' represents Reliance Industries Limited on the NSE (National Stock Exchange of India)
# - The data is downloaded for the time range from December 31, 2020, to December 20, 2024
df_r = yf.download('RELIANCE.NS', start='2020-12-31', end='2024-12-20')

# Resetting the DataFrame index to move the default date index to a regular column
# This makes it easier to manipulate and analyze the data using pandas
df_r.reset_index(inplace=True)

# Displaying the first few rows of the DataFrame to confirm the data was downloaded correctly
# The DataFrame includes columns like 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', and 'Volume'
df_r


[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
0,2020-12-31,894.531982,906.517334,891.513119,898.226723,18781189
1,2021-01-01,895.523254,899.803711,893.045057,895.748507,10015175
2,2021-01-04,897.032654,900.659759,886.736937,898.947600,24513534
3,2021-01-05,885.880859,893.765955,881.329980,887.187504,24123091
4,2021-01-06,862.518372,885.835720,858.418117,885.790670,46401468
...,...,...,...,...,...,...
976,2024-12-13,1272.849976,1275.199951,1239.599976,1260.000000,28630222
977,2024-12-16,1268.300049,1281.000000,1266.550049,1275.000000,9486781
978,2024-12-17,1245.300049,1263.900024,1242.800049,1261.050049,17462791


In [ ]:
def generate_atr_signals(data, atr_window=7, multiplier=1.5, volatility_threshold=25, volatility_threshold_low=25):
    """
    Generate trading signals based on the ATR (Average True Range).

    Parameters:
    data (pd.DataFrame): DataFrame containing 'High', 'Low', and 'Close' columns for stock data.
    atr_window (int): The look-back period used to calculate the ATR.
    multiplier (float): A multiplier used for defining stop-loss and take-profit levels.
    volatility_threshold (float): Threshold to identify high-volatility trades.
    volatility_threshold_low (float): Threshold to identify low-volatility trades.

    Returns:
    pd.DataFrame: DataFrame with additional columns for ATR and volatility flags.
    """
    # Calculate the True Range (TR) for each row in the data
    # TR is the maximum value among the following:
    # - Difference between High and Low prices
    # - Absolute difference between High and previous Close prices
    # - Absolute difference between Low and previous Close prices
    high_low = data['High'] - data['Low']
    high_close = abs(data['High'] - data['Close'].shift(1))
    low_close = abs(data['Low'] - data['Close'].shift(1))
    data['TR'] = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    # Calculate the Average True Range (ATR) as a rolling mean of TR
    # ATR gives a smoothed measure of the price volatility over a defined period (atr_window)
    data['ATR'] = data['TR'].rolling(window=atr_window).mean()

    # Identify high-volatility periods where ATR exceeds the specified threshold
    data['High_Volatility'] = data['ATR'] > volatility_threshold

    # Identify low-volatility periods where ATR falls below the specified threshold
    data['Low_Volatility'] = data['ATR'] < volatility_threshold_low

    # Remove the intermediate 'TR' column for a cleaner dataset
    data.drop(columns=['TR'], inplace=True)

    # Return the updated DataFrame with ATR and volatility indicators
    return data


In [ ]:
df_r =generate_atr_signals(df_r)
df_r

Price,Date,Close,High,Low,Open,Volume,ATR,High_Volatility,Low_Volatility
Ticker,,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,,,
0,2020-12-31,894.531982,906.517334,891.513119,898.226723,18781189,NaN,False,False
1,2021-01-01,895.523254,899.803711,893.045057,895.748507,10015175,NaN,False,False
2,2021-01-04,897.032654,900.659759,886.736937,898.947600,24513534,NaN,False,False
3,2021-01-05,885.880859,893.765955,881.329980,887.187504,24123091,NaN,False,False
4,2021-01-06,862.518372,885.835720,858.418117,885.790670,46401468,NaN,False,False
...,...,...,...,...,...,...,...,...,...
976,2024-12-13,1272.849976,1275.199951,1239.599976,1260.000000,28630222,20.242850,False,True
977,2024-12-16,1268.300049,1281.000000,1266.550049,1275.000000,9486781,18.907139,False,True
978,2024-12-17,1245.300049,1263.900024,1242.800049,1261.050049,17462791,20.564279,False,True


In [ ]:
# List of columns you want to keep
columns_to_keep = ['Symbol  ', 'Date  ', 'Expiry  ','Strike Price  ','LTP  ','Underlying Value  ']

# Keep only the specified columns
df_call = df_call[columns_to_keep]
df_put = df_put[columns_to_keep]

NameError: name 'df_call' is not defined

In [ ]:
df = pd.merge(df_put, df_call, on=['Symbol  ','Date  ', 'Expiry  ', 'Strike Price  ','Underlying Value  '], how='inner')

In [ ]:

df.rename(columns={'Expiry  ': '[EXPIRE_DATE]', 'LTP  _x': '[P_LAST]','LTP  _y': '[C_LAST]','Underlying Value  ': ' [UNDERLYING_LAST]','Strike Price  ': '[STRIKE]','Date  ': '[QUOTE_DATE]'}, inplace=True)

In [ ]:
# Replace '-' with 0 in the entire dataframe
df.replace('-', 0, inplace=True)

In [ ]:
df

In [ ]:
# Assuming 'df' has a 'QUOTE_DATE' column
df.set_index('[QUOTE_DATE]', inplace=True)


In [ ]:
mean_strike = df['[STRIKE]'].mean()
df['[STRIKE]'] = df['[STRIKE]'].apply(lambda x: x * 2 if x < mean_strike else x)

In [ ]:
mean_strike = df[' [UNDERLYING_LAST]'].mean()
df[' [UNDERLYING_LAST]'] = df[' [UNDERLYING_LAST]'].apply(lambda x: x * 2 if x < mean_strike else x)

In [ ]:
options_data = df

In [ ]:
options_data.rename(columns = {'[UNDERLYING_LAST]':' [UNDERLYING_LAST]'},inplace = True)


In [ ]:
options_data.rename(columns = {'[C_LAST]':' [C_LAST]'},inplace = True)

In [ ]:
options_data.rename(columns = {'[C_LAST]':' [C_LAST]'},inplace = True)

In [ ]:
options_data.rename(columns = {'[P_LAST]':' [P_LAST]'},inplace = True)

In [ ]:
import pandas as pd

# Convert to datetime format
df_r['Date_'] = pd.to_datetime(df_r['Date'])
options_data.index = pd.to_datetime(options_data.index)

# Flatten multi-level columns in df (if necessary)
if isinstance(df_r.columns, pd.MultiIndex):
    df_r.columns = ['_'.join(col).strip() for col in df_r.columns.values]

# Reset the index of options_data
options_data_reset = options_data.reset_index()

# Perform the merge
merged_data = options_data_reset.merge(
    df_r[['Date_', 'High_Volatility_', 'Low_Volatility_']],
    left_on='[QUOTE_DATE]',  # Use reset column
    right_on='Date_',         # Match with df_r's Date column
    how='left'               # Left join to preserve options_data rows
)

# Drop the redundant 'Date' column from the merged data
merged_data.drop(columns=['Date_'], inplace=True)

# Restore the index to [QUOTE_DATE]
merged_data.set_index('[QUOTE_DATE]', inplace=True)

# Display the resulting dataframe
print(merged_data)


In [ ]:
options_data = merged_data

In [ ]:
# Set the strike multiple
strike_multiple = 40

# Set the ATM strike price
options_data['ATM'] = strike_multiple * \
    (round(options_data[' [UNDERLYING_LAST]'] / strike_multiple))

# Ensure the index is in datetime format
options_data.index = pd.to_datetime(options_data.index)

# # Store the day of the week in the column 'day_of_week'
# options_data['day_of_week'] = options_data.index.day_name()



options_data['condition_1'] = options_data.High_Volatility_

# Check whether the days till expiry is more than 5
options_data['condition_2'] = options_data.Low_Volatility_

# Check if both conditions are met
options_data['signal'] = np.where(
    options_data['condition_1'] , 1, np.where(
    options_data['condition_2'] , -1, 1))

# options_data['signal'] = np.where(
#     options_data['condition_2'] , -1,1 )

# Drop any missing values
options_data = options_data.dropna()

# Print first 5 rows of the dataframe
options_data.head()

In [ ]:
# Create dataframes for round trips, storing trades, and mtm
round_trips_details = pd.DataFrame()
trades = pd.DataFrame()
mark_to_market = pd.DataFrame()

# Function for calculating mtm


def add_to_mtm(mark_to_market, option_strategy, trading_date):
    option_strategy['Date'] = trading_date
    mark_to_market = pd.concat([mark_to_market, option_strategy])
    return mark_to_market

# Function for fetching premium


def get_premium(options_strategy, options_data):

    # Get the premium for call option
    if options_strategy['Option Type'] == "CE":
        return options_data[' [C_LAST]']

    # Get the premium for put option
    elif options_strategy['Option Type'] == "PE":
        return options_data[' [P_LAST]']

# Function for setting up a straddle


def setup_straddle(options_data, direction='short'):

    # Create a dataframe to store the straddle
    straddle = pd.DataFrame()

    # CE and PE legs of the straddle
    straddle['Option Type'] = ['CE', 'PE']

    # Create the straddle at ATM
    straddle['Strike Price'] = options_data.ATM[0]

    # Sell positions for both CE and PE legs in case of a short straddle
    straddle['position'] = 1

    # Get the premiums for the two option legs of the short straddle
    straddle['premium'] = straddle.apply(
        lambda r: get_premium(r, options_data), axis=1)

    # Multiply the position by -1 in case of a long straddle
    if direction == 'long':
        straddle['position'] *= -1

    # Get the premiums for the two option legs of the long straddle
    straddle['premium'] = straddle.apply(
        lambda r: get_premium(r, options_data), axis=1)

    return straddle


# Initialise current position, number of trades and cumulative pnl to 0
current_position = 0
trade_num = 0
cum_pnl = 0
counter = 0

# Set exit flag to False
exit_flag = False

# Set start date for backtesting
start_date = options_data.index[0]

In [ ]:
config = {
    'stop_loss_percentage': 60,
    'take_profit_percentage': 20,
}

In [ ]:
for i in options_data.loc[start_date:].index.unique():

    # Today's options data
    today_data = options_data.loc[i]

    if current_position == 1:
        # Update the counter
        counter = counter + 1

        # Setup strike data
        setup_strike_data = today_data[today_data['[STRIKE]'] == setup_strike]

        # Update net premium
        straddle['premium'] = straddle.apply(
            lambda r: get_premium(r, setup_strike_data), axis=1)



        # Ensure net_premium is a float, handle missing or non-numeric values
        straddle['premium'] = pd.to_numeric(straddle['premium'], errors='coerce').fillna(0)

        net_premium = (straddle.position * straddle.premium).sum()
        net_premium = round(net_premium, 1)

        # Update mark_to_market dataframe
        mark_to_market = add_to_mtm(mark_to_market, straddle, i)

        # Exit the trade if any of the exit condition is met
        if counter == 5:
            exit_type = 'Rebalance'
            exit_flag = True

        elif net_premium < sl:
            exit_type = 'SL'
            exit_flag = True

        elif net_premium > tp:
            exit_type = 'TP'
            exit_flag = True

        if exit_flag:
            # Append the trades dataframe
            trades['exit_date'] = i
            trades['exit_type'] = exit_type
            trades['exit_price'] = straddle.premium

            # Calculate net premium at exit
            net_premium_exit = round((straddle.position * straddle.premium).sum(), 1)

            # Calculate net premium on entry
            entry_net_premium = pd.to_numeric(trades.position * trades.entry_price).sum()

            # Calculate pnl for the trade
            trade_pnl = round(net_premium_exit - entry_net_premium, 1)
            trades['PnL'] = trade_pnl

            # Add the trade logs to round trip details
            round_trips_details = pd.concat([round_trips_details, trades])

            # Calculate cumulative pnl
            cum_pnl += trade_pnl
            cum_pnl = round(cum_pnl, 2)

            # Print trade details
            print(f"Trade No: {trade_num} | Exit Type: {exit_type} | Date: {i} | Premium: {net_premium_exit} | Trade PnL: {trade_pnl} | Cum PnL: {cum_pnl}")

            # Update current position to 0
            current_position = 0

            # Reset counter
            counter = 0

            # Set exit flag to false
            exit_flag = False

    if (current_position == 0) and today_data['signal'][0] == 1:
        # ATM Strike data
        setup_strike_data = today_data[today_data['ATM'] == today_data['[STRIKE]']]

        # Setup straddle
        straddle = setup_straddle(setup_strike_data, direction="short")

        # Straddle setup strike
        setup_strike = today_data['ATM'][0]

        # Populate the trades dataframe
        trades = straddle.copy()
        trades['entry_date'] = i
        trades.rename(columns={'premium': 'entry_price'}, inplace=True)

        # Ensure position and premium are numeric
        straddle.position = pd.to_numeric(straddle.position, errors='coerce').fillna(0)
        straddle.premium = pd.to_numeric(straddle.premium, errors='coerce').fillna(0)

        # Calculate net premium
        net_premium = round((straddle.position * straddle.premium).sum(), 1)

        # Compute SL and TP for the trade
        premium_sign = np.sign(net_premium)
        sl = net_premium * (1 - config['stop_loss_percentage']*premium_sign/100)
        tp = net_premium * (1 + config['take_profit_percentage']*premium_sign/100)

        # Update current position to 1
        current_position = 1

        # Update mark_to_market dataframe
        mark_to_market = add_to_mtm(mark_to_market, straddle, i)

        # Increase number of trades by 1
        trade_num += 1
        print("-"*30)

        # Print trade details
        print(f"Trade No: {trade_num} | Entry | Date: {i} | Premium: {net_premium} | Trade PnL: 0 | Cum PnL: {cum_pnl}")


In [ ]:
# Round trip details
round_trips_details.head()

In [ ]:
# MTM details
mark_to_market.head()